<a href="https://colab.research.google.com/github/NguyenAnhKietUIT/IS254.O11/blob/main/%5BDSS_Nhung%5D_Downloading_and_Parsing_Wikipedia_Articles_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import Library

In [ ]:
import requests

# Parsing HTML
from bs4 import BeautifulSoup

# File system management
import os

### Searching for Wikipedia Download

In [ ]:
base_url = 'https://dumps.wikimedia.org/enwiki/'
index = requests.get(base_url).text
soup_index = BeautifulSoup(index, 'html.parser')

# Find the links that are dates of dumps
dumps = [a['href'] for a in soup_index.find_all('a') if
         a.has_attr('href')]
dumps

['../',
 '20230801/',
 '20230820/',
 '20230901/',
 '20230920/',
 '20231001/',
 '20231020/',
 '20231101/',
 'latest/']

Tìm kiếm nội dung HTML của trang web cho bản sao lưu được tạo ra vào ngày 01 tháng 08.

In [ ]:
dump_url = base_url + '20231020/'

# Retrieve the html
dump_html = requests.get(dump_url).text
dump_html[:10]

'<!DOCTYPE '

### Deciding What to Download

In [ ]:
# Convert to a soup
soup_dump = BeautifulSoup(dump_html, 'html.parser')

# Find li elements with the class file
soup_dump.find_all('li', {'class': 'file'}, limit = 10)[:4]

[<li class="file"><a href="/enwiki/20231020/enwiki-20231020-pages-articles-multistream.xml.bz2">enwiki-20231020-pages-articles-multistream.xml.bz2</a> 21.0 GB</li>,
 <li class="file"><a href="/enwiki/20231020/enwiki-20231020-pages-articles-multistream-index.txt.bz2">enwiki-20231020-pages-articles-multistream-index.txt.bz2</a> 242.9 MB</li>,
 <li class="file"><a href="/enwiki/20231020/enwiki-20231020-pages-articles-multistream1.xml-p1p41242.bz2">enwiki-20231020-pages-articles-multistream1.xml-p1p41242.bz2</a> 261.2 MB</li>,
 <li class="file"><a href="/enwiki/20231020/enwiki-20231020-pages-articles-multistream-index1.txt-p1p41242.bz2">enwiki-20231020-pages-articles-multistream-index1.txt-p1p41242.bz2</a> 221 KB</li>]

limit to only the files containing pages-articles to get only the recent versions of the articles.

In [ ]:
files = []

# Search through all files
for file in soup_dump.find_all('li', {'class': 'file'}):
    text = file.text
    # Select the relevant files
    if 'pages-articles' in text:
        files.append((text.split()[0], text.split()[1:]))

files

[('enwiki-20231020-pages-articles-multistream.xml.bz2', ['21.0', 'GB']),
 ('enwiki-20231020-pages-articles-multistream-index.txt.bz2', ['242.9', 'MB']),
 ('enwiki-20231020-pages-articles-multistream1.xml-p1p41242.bz2',
  ['261.2', 'MB']),
 ('enwiki-20231020-pages-articles-multistream-index1.txt-p1p41242.bz2',
  ['221', 'KB']),
 ('enwiki-20231020-pages-articles-multistream2.xml-p41243p151573.bz2',
  ['348.7', 'MB']),
 ('enwiki-20231020-pages-articles-multistream-index2.txt-p41243p151573.bz2',
  ['637', 'KB']),
 ('enwiki-20231020-pages-articles-multistream3.xml-p151574p311329.bz2',
  ['378.9', 'MB']),
 ('enwiki-20231020-pages-articles-multistream-index3.txt-p151574p311329.bz2',
  ['819', 'KB']),
 ('enwiki-20231020-pages-articles-multistream4.xml-p311330p558391.bz2',
  ['422.1', 'MB']),
 ('enwiki-20231020-pages-articles-multistream-index4.txt-p311330p558391.bz2',
  ['1.3', 'MB']),
 ('enwiki-20231020-pages-articles-multistream5.xml-p558392p958045.bz2',
  ['452.5', 'MB']),
 ('enwiki-2023102

Phân vùng tệp, lựa chọn tệp có đuôi ``` xml-p ```





In [ ]:
files_to_download = [file[0] for file in files if '.xml-p' in file[0]]
files_to_download[-5:]

['enwiki-20231020-pages-articles27.xml-p68475910p69975909.bz2',
 'enwiki-20231020-pages-articles27.xml-p69975910p71475909.bz2',
 'enwiki-20231020-pages-articles27.xml-p71475910p72975909.bz2',
 'enwiki-20231020-pages-articles27.xml-p72975910p74475909.bz2',
 'enwiki-20231020-pages-articles27.xml-p74475910p75105778.bz2']

### Download Wikipedia Data

In [ ]:
!pip install keras

In [ ]:
import sys
from keras.utils import get_file

keras_home = '/content/sample_data/dataset/'
print(keras_home)

/content/sample_data/dataset/


In [ ]:
for file in files_to_download:
    path = keras_home + file

    # Check to see if the path exists (if the file is already downloaded)
    if not os.path.exists(keras_home + file):
        # If not, download the file
        data_paths.append(get_file(file, dump_url))

In [ ]:
data_paths = []
file_info = []

# Iterate through each file
for file in files_to_download:
    path = keras_home + file

    # Check to see if the path exists (if the file is already downloaded)
    if not os.path.exists(keras_home + file):
        print('Downloading')
        # If not, download the file
        data_paths.append(get_file(file, dump_url))
        # Find the file size in MB
        file_size = os.stat(path).st_size / 1e6

        # Find the number of articles
        file_articles = int(file.split('p')[-1].split('.')[-2]) - int(file.split('p')[-2])
        file_info.append((file, file_size, file_articles))

    # If the file is already downloaded find some information
    else:
        data_paths.append(path)
        # Find the file size in MB
        file_size = os.stat(path).st_size / 1e6

        # Find the number of articles
        file_number = int(file.split('p')[-1].split('.')[-2]) - int(file.split('p')[-2])
        file_info.append((file.split('-')[-1], file_size, file_number))

Downloading


FileNotFoundError: ignored

In [ ]:
sorted(file_info, key = lambda x: x[1], reverse = True)[:5]

[]

In [ ]:
sorted(file_info, key = lambda x: x[2], reverse = True)[:5]

[]

In [ ]:
print(f'There are {len(file_info)} partitions.')

There are 0 partitions.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
file_df = pd.DataFrame(file_info, columns = ['file', 'size (MB)', 'articles']).set_index('file')
file_df['size (MB)'].plot.bar(color = 'red', figsize = (12, 6));

TypeError: ignored